In [1]:
import pandas as pd
import numpy as np
import openai
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from utils_llm import get_llm_output, get_llm_embedding  



False


In [2]:
from openai import OpenAI

client = OpenAI(api_key = "YOUR_OWN_API_KEY"
)

def get_llm_output(prompt: str, model: str, cache=True, system_prompt=None, history=[], max_tokens=256) -> str:
    if "gpt" in model:
        completion = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=max_tokens
        )
        return completion.choices[0].message.content

In [3]:
try:
    response = openai.Model.list()
    print("API Key is working!")
except Exception as e:
    print(f"Error: {e}")

Error: 

You tried to access openai.Model, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742



In [4]:
#checking current working directory
current_directory = os.getcwd()
print(current_directory)

/Users/doga/Desktop/VibeCheck/jupyter_notebook


In [5]:
df = pd.read_csv('/Users/doga/Desktop/VibeCheck/data/cnndm_with_pref.csv') 

In [6]:
def generate_outputs(row, models):
    """
    Generate outputs from Model A and Model B for the given row.

    Args:
        row (pd.Series): A row from the DataFrame containing the question.
        models (list): A list of model names.

    Returns:
        tuple: A tuple containing outputs from Model A and Model B.
    """
    prompt = row['question']  # col with the user prompt
    output_A = get_llm_output(prompt, models[0]) 
    output_B = get_llm_output(prompt, models[1])
    return output_A, output_B

In [7]:
def get_embeddings(outputs, model):
    """
    Placeholder function to extract embeddings from model outputs.
    Replace this with your actual embedding extraction logic.

    Args:
        outputs (list): List of model outputs.

    Returns:
        np.ndarray: Array of embeddings.
    """
    embeddings = []
    for output in outputs:
        embedding = get_llm_embedding(output, model)  # Get embedding for each output
        embeddings.append(embedding)  
    return np.array(embeddings)

In [8]:
def prepare_dataset(df, models):
    """
    Prepare the dataset for training the classifier.

    Args:
        df (pd.DataFrame): DataFrame containing the outputs from both models.
        models (list): A list of model names.

    Returns:
        tuple: Features and labels for the classifier.
    """
    # Generate outputs from both models
    df['output_A'], df['output_B'] = zip(*df.apply(lambda row: generate_outputs(row, models), axis=1))
    
    # Extract embeddings 
    embeddings_A = get_embeddings(df['output_A'])
    embeddings_B = get_embeddings(df['output_B'])

    # Features and labels
    X = np.concatenate((embeddings_A, embeddings_B), axis=0)
    y = np.array([0] * len(embeddings_A) + [1] * len(embeddings_B))  # 0 for Model A, 1 for Model B

    return X, y


In [9]:
def train_classifier(X, y):
    """
    Train a classifier on the provided features and labels.

    Args:
        X (np.ndarray): Features for training.
        y (np.ndarray): Labels for training.

    Returns:
        tuple: Trained classifier and its accuracy on the test set.
    """
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    clf = MLPClassifier(hidden_layer_sizes=(100,), max_iter=500)
    clf.fit(X_train, y_train)

    # Evaluate
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    return clf, accuracy

In [10]:
if __name__ == "__main__":
    
    df = pd.read_csv("/Users/doga/Desktop/VibeCheck/data/cnndm_with_pref.csv")
    models = ["gpt-4", "gpt-3.5-turbo"]

    # Prepare dataset
    X, y = prepare_dataset(df, models)

    # Train classifier
    clf, accuracy = train_classifier(X, y)
    print(f'Classifier Accuracy: {accuracy}')

KeyboardInterrupt: 